# KEY NOTE

This notebook is guide to core pyspark datatype RDD. Since most technologies and industries now a days are working on cloud, it is good to have knowledge on how to apply ML and Datascience when it comes to "Big Data". I will be modifying the code and content already available so that we can use it with simple dataset like "Titanic" which is the hello world of ML.

These are just my personal notes. I am sharing these so that it helps others too, who are trying to learn the similar concepts. Any Feedback is appreciated.

If you havent checked the Spark DataFrame notebook, please feel free to do so [here](https://www.kaggle.com/amritvirsinghx/scalable-data-science-pyspark-nb1)

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Notebook Navigation</h3>

[1. What is Spark?](#1)     
[2. Using Spark in Python](#2)    
[3. Spark Core RDD](#3)    
[4. Creating RDDs](#4)    
&nbsp;&nbsp;&nbsp;&nbsp;[a. Creating an Empty RDD](#4a)   
&nbsp;&nbsp;&nbsp;&nbsp;[b. Creating RDD with Partitions](#4b)       
&nbsp;&nbsp;&nbsp;&nbsp;[c. Checking Number of Partitions](#4c)   
&nbsp;&nbsp;&nbsp;&nbsp;[d. Collecting the Data with Partition](#4d)   
&nbsp;&nbsp;&nbsp;&nbsp;[e. Setting Name of RDD](#4e)       
&nbsp;&nbsp;&nbsp;&nbsp;[f. Using Range to Create RDD](#4f)  
[5. Ordering and Repartitioning RDD](#5)  
&nbsp;&nbsp;&nbsp;&nbsp;[a. Fetch Ordered Elements](#5a)   
&nbsp;&nbsp;&nbsp;&nbsp;[b. Repartitions and Coalesce](#5b)       
[6. Saving and Debugging](#6)     
&nbsp;&nbsp;&nbsp;&nbsp;[a. Saving RDD to a Text File](#6a)   
&nbsp;&nbsp;&nbsp;&nbsp;[b. Checking Lineage of RDD](#6b)  
[7. Performing Operations on RDD](#7)         
&nbsp;&nbsp;&nbsp;&nbsp;[a. Performing Reduce](#7a)   
&nbsp;&nbsp;&nbsp;&nbsp;[b. Creating User Defined Functions (UDF)](#7b)    
&nbsp;&nbsp;&nbsp;&nbsp;[c. Applying Function as Filter](#7c)      
&nbsp;&nbsp;&nbsp;&nbsp;[d. Creating Flat Maps](#7d)     
&nbsp;&nbsp;&nbsp;&nbsp;[e. Performing Joins on RDD](#7e)       
&nbsp;&nbsp;&nbsp;&nbsp;[f. Listing and Replication](#7f)      
&nbsp;&nbsp;&nbsp;&nbsp;[g. Grouping on RDD](#7g)     
&nbsp;&nbsp;&nbsp;&nbsp;[h. Reducing by Key and Functions](#7h)       
[8. Epilogue](#8)   

<a id="1"></a>
## 1. What is Spark?

Spark is a platform for cluster computing. Spark lets you spread data and computations over clusters with multiple nodes (think of each node as a separate computer). Splitting up your data makes it easier to work with very large datasets because each node only works with a small amount of data.

As each node works on its own subset of the total data, it also carries out a part of the total calculations required, so that both data processing and computation are performed in parallel over the nodes in the cluster. It is a fact that parallel computation can make certain types of programming tasks much faster.

However, with greater computing power comes greater complexity.

If you are deciding whether or not Spark is the best solution for your problem you can consider questions like:
* Is my data too big to work with on a single machine?
* Can my calculations be easily parallelized?

![Spark Logo](https://www.lwindia.com/images/Cloudera-Landing-Page-Banner.jpg)

<a id="2"></a>
## 2. Using Spark in Python

The first step in using Spark is connecting to a cluster.

In practice, the cluster will be hosted on a remote machine that's connected to all other nodes. There will be one computer, called the master that manages splitting up the data and the computations. The master is connected to the rest of the computers in the cluster, which are called worker. The master sends the workers data and calculations to run, and they send their results back to the master.

When just getting started with Spark it's simpler to just run a cluster locally. Thus for now, instead of connecting to another computer, all computations will be run on Kaggle's servers in a simulated cluster.

Creating the connection is as simple as creating an instance of the SparkContext class. The class constructor takes a few optional arguments that allow us to specify the attributes of the cluster we're connecting to.

In [28]:
!pip install pyspark

<a id="3"></a>
# 3. Spark Core RDD

RDD stands for Resilient Distributed Dataset, these are the elements that run and operate on multiple nodes to do parallel processing on a cluster. RDDs are immutable elements, which means once you create an RDD you cannot change it. RDDs are fault tolerant as well, hence in case of any failure, they recover automatically. You can apply multiple operations on these RDDs to achieve a certain task.

![RDD](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1991843%2Fece9429ac833006eda20f0aabede3860%2FCapture.JPG?generation=1601101924589580&alt=media)


To apply operations on these RDD's, there are two ways −

- Transformation
- Action

**Transformation** − These are the operations, which are applied on a RDD to create a new RDD. Filter, groupBy and map are the examples of transformations.

**Action** − These are the operations that are applied on RDD, which instructs Spark to perform computation and send the result back to the driver.

We can create initialize spark core RDD context as follows:

In [29]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


<a id="4"></a>
## 4. Creating RDDs
We can RDDs in many different ways lets go through few of them

<a id="4a"></a>
### a. Creating an Empty RDD

In [30]:
rdd=sc.emptyRDD()
rdd.isEmpty()

True

In [31]:
a = sc.parallelize([])
a.isEmpty()

True

<a id="4b"></a>
### b. Creating RDD with Partitions

In [32]:
mixRDD =sc.parallelize([True, [11,22,33,44,55], (10,20,30,40,50)], 3)
mixRDD.collect()

[True, [11, 22, 33, 44, 55], (10, 20, 30, 40, 50)]

<a id="4c"></a>
### c. Checking Number of Partitions

In [33]:
mixRDD.getNumPartitions()

3

<a id="4d"></a>
### d. Collecting the Data with Partitions

In [34]:
mixRDD.glom().collect()

[[True], [[11, 22, 33, 44, 55]], [(10, 20, 30, 40, 50)]]

<a id="4e"></a>
### e. Setting Name of RDD

In [35]:
mixRDD.setName('MyRDD')

MyRDD ParallelCollectionRDD[43] at readRDDFromFile at PythonRDD.scala:289

<a id="4f"></a>
### f. Using Range to Create RDD

In [36]:
a= sc.parallelize(range(1,1000), 4)
print(a.getNumPartitions())
print("\n")
print(a.glom().take(1))
print("\n")
print(a.glom().max()) #to get max partitions data means last one partition(3)
print("\n")
print(a.glom().min()) #to get min partitions data means first one partition(0)
print("\n")

4


[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 22

Expand the output to see the result

<a id="5"></a>
## 5. Ordering and Repartitioning RDD

<a id="5a"></a>
### a. Fetch Ordered Elements

In [37]:
b =sc.parallelize([12,21,23,43,1,22,11,45,56])
b.takeOrdered(4, key=lambda x: -x)

[56, 45, 43, 23]

<a id="5b"></a>
### b. Repartitions and Coalesce

In [38]:
c=sc.parallelize(range(1,1000), 5)
c.getNumPartitions()

5

We can increase or decrease number of partitions with repartition method

In [39]:
d= c.repartition(7)
d.getNumPartitions()

7

In [40]:
e= d.repartition(4)
e.getNumPartitions()

4

With coalesce we cannot increase the partitions

In [41]:
e.coalesce(6)
e.getNumPartitions()

4

In [42]:
e.coalesce(2)
e.getNumPartitions()

4

<a id="6"></a>
## 6. Saving and Debugging

<a id="6a"></a>
### a. Saving RDD to a Text File

In [77]:
e.saveAsTextFile("./sampletext1")

The output will be saved in kaggle's output directory, notice how the file is saved in partitions as per the number of partitions of RDD

<a id="6b"></a>
### b. Checking Lineage of RDD

In [44]:
e.toDebugString()

b'(4) MapPartitionsRDD[60] at coalesce at NativeMethodAccessorImpl.java:0 []\n |  CoalescedRDD[59] at coalesce at NativeMethodAccessorImpl.java:0 []\n |  ShuffledRDD[58] at coalesce at NativeMethodAccessorImpl.java:0 []\n +-(7) MapPartitionsRDD[57] at coalesce at NativeMethodAccessorImpl.java:0 []\n    |  MapPartitionsRDD[56] at coalesce at NativeMethodAccessorImpl.java:0 []\n    |  CoalescedRDD[55] at coalesce at NativeMethodAccessorImpl.java:0 []\n    |  ShuffledRDD[54] at coalesce at NativeMethodAccessorImpl.java:0 []\n    +-(5) MapPartitionsRDD[53] at coalesce at NativeMethodAccessorImpl.java:0 []\n       |  PythonRDD[52] at RDD at PythonRDD.scala:53 []\n       |  ParallelCollectionRDD[51] at readRDDFromFile at PythonRDD.scala:289 []'

Ouput DAG can be viewed on Spark console on the port which is configured. A sample DAG is as follows:

![DAG](https://1.bp.blogspot.com/-OYuEUWP8UZo/XZQ4ZOF8ApI/AAAAAAAADGc/_nwzjUP8BHIEFr0FNHy3Vt55xeJYBsfdwCLcBGAsYHQ/s640/sp_1.png)

To know more about DAGs follow this [link](https://databricks.com/blog/2015/06/22/understanding-your-spark-application-through-visualization.html)

<a id="7"></a>
## 7. Performing Operations on RDDs
We can perform two kind of operations on RDD:

### i. Transformation
Transformations are kind of operations which will transform your RDD data from one form to another. And when you apply this operation on any RDD, you will get a new RDD with transformed data (RDDs in Spark are immutable). Operations like map, filter, flatMap are transformations.

Now there is a point to be noted here and that is when you apply the transformation on any RDD it will not perform the operation immediately. It will create a DAG(Directed Acyclic Graph) using the applied operation, source RDD and function used for transformation. And it will keep on building this graph using the references till you apply any action operation on the last lined up RDD. That is why the **transformation in Spark are lazy**.
Spark has certain operations which can be performed on RDD. An operation is a method, which can be applied on a RDD to accomplish certain task. RDD supports two types of operations, which are Action and Transformation. An operation can be something as simple as sorting, filtering and summarizing data

Further transformations are of two types:

- Narrow transformation

In Narrow transformation, all the elements that are required to compute the records in single partition live in the single partition of parent RDD. A limited subset of partition is used to calculate the result. Narrow transformations are the result of map(), filter().

- Wide transformation

In wide transformation, all the elements that are required to compute the records in the single partition may live in many partitions of parent RDD. The partition may live in many partitions of parent RDD. Wide transformations are the result of groupbyKey and reducebyKey.

### ii. Actions
Transformations create RDDs from each other, but when we want to work with the actual dataset, at that point action is performed. When the action is triggered after the result, new RDD is not formed like transformation. Thus, actions are RDD operations that give non-RDD values. The values of action are stored to drivers or to the external storage system. It brings laziness of RDD into motion.

Spark drivers and external storage system store the value of action. It brings laziness of RDD into motion.
An action is one of the ways of sending data from Executer to the driver. Executors are agents that are responsible for executing a task. While the driver is a JVM process that coordinates workers and execution of the task

<a id="7a"></a>
### a. Performing Reduce

In [45]:
i =sc.parallelize([1,9,5,6,7,8])
i.reduce(lambda a,b:a+b)

36

In [46]:
i.reduce(lambda a,b:a*b)

15120

<a id="7b"></a>
### b. Creating User Defined Functions (UDF)

In [47]:
word =['how', 'are', 'you', 'hey', 'hi']
wordRDD=sc.parallelize(word)
wordRDD.collect()

['how', 'are', 'you', 'hey', 'hi']

In [48]:
# defining functions
def start_h(word):
    return word[0].lower().startswith('h')

<a id="7c"></a>
### c. Applying Function as Filter

In [49]:
wordRDD.filter(start_h).collect() # it will return all words starts with 'h'

['how', 'hey', 'hi']

In [50]:
def toUpper(s):
    return s.upper()

In [51]:
x=['navin', 'kumar', 'pal']
x1=sc.parallelize(x)
k3=x1.map(toUpper)
k3.collect()

['NAVIN', 'KUMAR', 'PAL']

In [52]:
k3=x1.map(lambda k:k.upper())
k3.collect()

['NAVIN', 'KUMAR', 'PAL']

Creating RDD from File

textFile method reads froom csv as well as text

In [53]:
demo = sc.textFile("/content/titanic-training-data.csv")
demo.collect()

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S',
 '5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S',
 '6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q',
 '7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S',
 '8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S',
 '9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S',
 '10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C',
 '11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7,G6,S',
 '12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,C103,S',
 '

<a id="7d"></a>
### d. Creating Flat Maps

In [54]:
demo.flatMap(lambda x:x.split(",")).map(lambda x: (x,1)).reduceByKey(lambda a,b:a+ b).collect()

[('Survived', 1),
 ('Pclass', 1),
 ('Sex', 1),
 ('Age', 1),
 ('Parch', 1),
 ('Ticket', 1),
 ('Fare', 1),
 ('1', 893),
 ('0', 1850),
 ('"Braund', 2),
 ('22', 28),
 ('', 866),
 ('S', 644),
 (' Mrs. John Bradley (Florence Briggs Thayer)"', 1),
 ('female', 314),
 ('71.2833', 1),
 ('C', 168),
 ('"Heikkinen', 1),
 ('26', 50),
 ('7.925', 18),
 ('4', 33),
 (' Mrs. Jacques Heath (Lily May Peel)"', 1),
 ('113803', 2),
 ('53.1', 5),
 ('373450', 1),
 ('"Moran', 3),
 (' Mr. James"', 7),
 ('54', 9),
 ('51.8625', 2),
 ('8', 12),
 ('"Palsson', 4),
 (' Master. Gosta Leonard"', 1),
 ('349909', 4),
 ('21.075', 4),
 ('9', 11),
 (' Mrs. Oscar W (Elisabeth Vilhelmina Berg)"', 1),
 ('11.1333', 3),
 ('10', 3),
 ('"Nasser', 2),
 ('14', 8),
 ('237736', 2),
 ('30.0708', 2),
 ('"Sandstrom', 2),
 (' Miss. Marguerite Rut"', 1),
 ('16.7', 2),
 ('12', 3),
 (' Miss. Elizabeth"', 2),
 ('113783', 1),
 ('C103', 1),
 ('"Saundercock', 1),
 ('20', 16),
 ('A/5. 2151', 1),
 ('"Andersson', 9),
 (' Mr. Anders Johan"', 1),
 (' M

Expand the output to see the results

In [55]:
t1=(['navin', 121223], ['kumar',3000])
t1rdd=sc.parallelize(t1,2)
print(t1rdd.collect())
print(t1rdd.getNumPartitions())
t1rdd.collectAsMap()

[['navin', 121223], ['kumar', 3000]]
2


{'navin': 121223, 'kumar': 3000}

<a id="7e"></a>
### e. Perfoming Joins on RDD

In [56]:
C=((201, "pune"), (301, "kol"),(201, "Mum"), (402, "Jaipur"),(505,"RTM"))
D=sc.parallelize(C)
A=((201, "navin"), (301, "kumar" ), (402, "pal"),(603,'kk'))
B=sc.parallelize(A)
tupleJoin=B.join(D)
tupleJoin.collect()

[(201, ('navin', 'pune')),
 (201, ('navin', 'Mum')),
 (301, ('kumar', 'kol')),
 (402, ('pal', 'Jaipur'))]

In [57]:
tupleLeftJoin=B.leftOuterJoin(D)
tupleLeftJoin.collect()

[(201, ('navin', 'pune')),
 (201, ('navin', 'Mum')),
 (301, ('kumar', 'kol')),
 (402, ('pal', 'Jaipur')),
 (603, ('kk', None))]

<a id="7f"></a>
### f. Listing and Replication

In [58]:
sc.parallelize([5,6,7]).map(lambda x:[x,x,x]).collect()

[[5, 5, 5], [6, 6, 6], [7, 7, 7]]

In [59]:
sc.parallelize([5,6,7]).map(lambda x:[[x,x,x],[x*x*x],[x+5]]).collect()

[[[5, 5, 5], [125], [10]], [[6, 6, 6], [216], [11]], [[7, 7, 7], [343], [12]]]

In [60]:
input1 = sc.parallelize(["apple", "banana", "pineapple"])
print(input1.collect())
input1.count()

['apple', 'banana', 'pineapple']


3

In [61]:
a=sc.parallelize([5,5,6,7,8,8,8,9]).countByValue()
a

defaultdict(int, {5: 2, 6: 1, 7: 1, 8: 3, 9: 1})

<a id="7g"></a>
### g. Grouping on RDD

In [65]:
i =sc.textFile("/content/titanic-training-data.csv",4)
i.collect()

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S',
 '5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S',
 '6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q',
 '7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S',
 '8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S',
 '9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S',
 '10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C',
 '11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7,G6,S',
 '12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,C103,S',
 '

In [66]:
j=i.map(lambda x:x.split(","))
k=j.map(lambda field:(field[5], field[1]))
k.collect()

[('Age', 'Survived'),
 ('male', '0'),
 ('female', '1'),
 ('female', '1'),
 ('female', '1'),
 ('male', '0'),
 ('male', '0'),
 ('male', '0'),
 ('male', '0'),
 ('female', '1'),
 ('female', '1'),
 ('female', '1'),
 ('female', '1'),
 ('male', '0'),
 ('male', '0'),
 ('female', '0'),
 ('female', '1'),
 ('male', '0'),
 ('male', '1'),
 ('female', '0'),
 ('female', '1'),
 ('male', '0'),
 ('male', '1'),
 ('female', '1'),
 ('male', '1'),
 ('female', '0'),
 ('female', '1'),
 ('male', '0'),
 ('male', '0'),
 ('female', '1'),
 ('male', '0'),
 ('male', '0'),
 ('female', '1'),
 ('female', '1'),
 ('male', '0'),
 ('male', '0'),
 ('male', '0'),
 ('male', '1'),
 ('male', '0'),
 ('female', '0'),
 ('female', '1'),
 ('female', '0'),
 ('female', '0'),
 ('male', '0'),
 ('female', '1'),
 ('female', '1'),
 ('male', '0'),
 ('male', '0'),
 ('female', '1'),
 ('male', '0'),
 ('female', '0'),
 ('male', '0'),
 ('male', '0'),
 ('female', '1'),
 ('female', '1'),
 ('male', '0'),
 ('male', '1'),
 ('female', '1'),
 ('male', 

In [67]:
L=k.groupByKey()
for j in L.collect():
    print([j[0],list(j[1])])

['Age', ['Survived']]
['female', ['1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '0', '0', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '1', '1', '1', '

Finally you can check the lineage

In [68]:
print(L.toDebugString())
L.getNumPartitions()

b'(4) PythonRDD[118] at collect at <ipython-input-67-eb89f7f79ba7>:2 []\n |  MapPartitionsRDD[117] at mapPartitions at PythonRDD.scala:160 []\n |  ShuffledRDD[116] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(4) PairwiseRDD[115] at groupByKey at <ipython-input-67-eb89f7f79ba7>:1 []\n    |  PythonRDD[114] at groupByKey at <ipython-input-67-eb89f7f79ba7>:1 []\n    |  /content/titanic-training-data.csv MapPartitionsRDD[112] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /content/titanic-training-data.csv HadoopRDD[111] at textFile at NativeMethodAccessorImpl.java:0 []'


4

<a id="7h"></a>
### h. Reducing by Key and Functions

In [69]:
# creating datframe
x=[('Designation', 'Salary'), ('AM', '50000'), ('AM', '50000'), ('SSE', '30000'), ('SSE', '30000'), ('Lead', '40000'), ('Lead', '35000'), ('ASE', '15000'), ('ASE', '15000'), ('SE', '22000'), ('SE', '25000'), ('SE', '25000'), ('ASE', '20000'), ('ASE', '18000'), ('ASE', '15000'), ('ASE', '18000')]
k=sc.parallelize(x)
k.collect()

[('Designation', 'Salary'),
 ('AM', '50000'),
 ('AM', '50000'),
 ('SSE', '30000'),
 ('SSE', '30000'),
 ('Lead', '40000'),
 ('Lead', '35000'),
 ('ASE', '15000'),
 ('ASE', '15000'),
 ('SE', '22000'),
 ('SE', '25000'),
 ('SE', '25000'),
 ('ASE', '20000'),
 ('ASE', '18000'),
 ('ASE', '15000'),
 ('ASE', '18000')]

In [70]:
k.reduceByKey(lambda x,y:int(int(x)+int(y))).collect()

[('SSE', 60000),
 ('Designation', 'Salary'),
 ('AM', 100000),
 ('Lead', 75000),
 ('ASE', 101000),
 ('SE', 72000)]

In [71]:
 k.reduceByKey(lambda x,y:int(int(x)+int(y))).sortByKey().collect()

[('AM', 100000),
 ('ASE', 101000),
 ('Designation', 'Salary'),
 ('Lead', 75000),
 ('SE', 72000),
 ('SSE', 60000)]

In [72]:
for i in k.collect():
    print (i)

('Designation', 'Salary')
('AM', '50000')
('AM', '50000')
('SSE', '30000')
('SSE', '30000')
('Lead', '40000')
('Lead', '35000')
('ASE', '15000')
('ASE', '15000')
('SE', '22000')
('SE', '25000')
('SE', '25000')
('ASE', '20000')
('ASE', '18000')
('ASE', '15000')
('ASE', '18000')


In [73]:
k.countByKey()

defaultdict(int,
            {'Designation': 1,
             'AM': 2,
             'SSE': 2,
             'Lead': 2,
             'ASE': 6,
             'SE': 3})

In [74]:
 k.countByValue()

defaultdict(int,
            {('Designation', 'Salary'): 1,
             ('AM', '50000'): 2,
             ('SSE', '30000'): 2,
             ('Lead', '40000'): 1,
             ('Lead', '35000'): 1,
             ('ASE', '15000'): 3,
             ('SE', '22000'): 1,
             ('SE', '25000'): 2,
             ('ASE', '20000'): 1,
             ('ASE', '18000'): 2})

In [75]:
 k.groupByKey().distinct().count()

6

In [76]:
k.sortByKey().collect()

[('AM', '50000'),
 ('AM', '50000'),
 ('ASE', '15000'),
 ('ASE', '15000'),
 ('ASE', '20000'),
 ('ASE', '18000'),
 ('ASE', '15000'),
 ('ASE', '18000'),
 ('Designation', 'Salary'),
 ('Lead', '40000'),
 ('Lead', '35000'),
 ('SE', '22000'),
 ('SE', '25000'),
 ('SE', '25000'),
 ('SSE', '30000'),
 ('SSE', '30000')]

<a id="8"></a>
## 8. Epilogue

These are some of the core operations that we perform on RDDs for further operations like querying and manipulating data we have RDD abstractions called dataframes, these are kind of similar to pd dataframes and can can also be convertedd back and forth to pandas dataframes, most data science opertations are performed with the use of Spark Dynamic frames.

To know more bout them checkout this [notebook](https://www.kaggle.com/amritvirsinghx/scalable-data-science-pyspark-nb1)

One last thing, if we want to ingest real time data and perform actions on streaming data we have something called spark streaming context. We ll cover it up going further, if the implementation is possible through kaggle notebooks.

Danke!